# Temporary Notebook: Extracting Initial Game Details Data

Scope:
- Games Released from 2018 to 2023
- Non-Indie Games (Games not released on Itch.io)

In [1]:
import pandas as pd
import os
import requests
import numpy as np

# create a .env file and store RAWG_TOKEN="<Your API Key>"
from dotenv import load_dotenv
load_dotenv()

RAWG_TOKEN = os.getenv('RAWG_TOKEN')


from utilities import (
    get_list_response,
    get_detail_response,
    transform_game_list_api,
    transform_game_detail_api,
    create_dictionary_from_df
)

### 1: Importing the list of ID and slug_name of all Games

In [2]:
raw_data_directory = os.path.join(os.getcwd(), "initial_upload_raw_data")

In [3]:
df_game_tag = pd.read_csv(os.path.join(raw_data_directory, "game_tag.csv"))

In [4]:
df_game_data = pd.read_csv(os.path.join(raw_data_directory, "game_data.csv"))

In [5]:
df_game_data[["id", "slug"]]

,id,slug
0,58175,god-of-war-2
1,29177,detroit-become-human
2,46889,monster-hunter-world-2
3,23585,far-cry-5
4,45969,warhammer-vermintide-2
...,...,...
66296,926947,modern-cannon-strike
66297,926877,card-brawl-dungeon-slayer
66298,926972,qbert-2023
66299,926914,3d-car-game-parking-jam


In [6]:
# Dictionary mapping each id to slug incase id fails
dct_id_slug = create_dictionary_from_df(df_game_data, "id", "slug")

In [7]:
game_list_id = df_game_data["id"].unique().tolist()
len(game_list_id)

66301

### 2: Extracting from Game Details

- Will definitely exceed API limit - So have to split into 5 different API calls

In [8]:
index_list = [
    [0 , 15000],
    [15000 , 30000],
    [30000 , 45000],
    [45000 , 60000],
    [60000 , 66301],
]

In [9]:
df_game_details_data = pd.DataFrame()
df_game_details_metacritic = pd.DataFrame()
df_game_details_developers = pd.DataFrame()
df_game_details_publishers = pd.DataFrame()
df_game_details_reactions = pd.DataFrame()

In [10]:
index_start = 30000
index_stop = 45000

In [17]:
for i in range(index_start, index_stop):
    game_id = game_list_id[i]

    game_detail_json = get_detail_response(RAWG_TOKEN, "games", game_id)

    if type(game_detail_json) == requests.models.Response:
        game_slug = dct_id_slug[game_id]
        game_detail_json = get_detail_response(RAWG_TOKEN, "games", game_slug)
    
    dct_game_details = transform_game_detail_api(game_detail_json)

    df_game_details_data = pd.concat([df_game_details_data, dct_game_details["game_details_data"]])
    df_game_details_metacritic = pd.concat([df_game_details_metacritic, dct_game_details["metacritic_platforms"]])
    df_game_details_developers = pd.concat([df_game_details_developers, dct_game_details["developers"]])
    df_game_details_publishers = pd.concat([df_game_details_publishers, dct_game_details["publishers"]])
    
    if "reactions" in dct_game_details.keys():
        df_game_details_reactions = pd.concat([df_game_details_reactions, dct_game_details["reactions"]])

    print(f"Done Extracting Game {i+1} / {index_stop}")

In [12]:
# to replace index_start with i incase of bug
print(i)

29999


### 3: Exporting the Data Extracted

In [16]:
df_game_details_data.to_csv(os.path.join(raw_data_directory, f"game_details_data_{index_start}_{i}.csv"), index=False)
df_game_details_metacritic.to_csv(os.path.join(raw_data_directory, f"game_details_metacritic_{index_start}_{i}.csv"), index=False)
df_game_details_developers.to_csv(os.path.join(raw_data_directory, f"game_details_developer_{index_start}_{i}.csv"), index=False)
df_game_details_publishers.to_csv(os.path.join(raw_data_directory, f"game_details_publisher_{index_start}_{i}.csv"), index=False)
df_game_details_reactions.to_csv(os.path.join(raw_data_directory, f"game_details_reaction_{index_start}_{i}.csv"), index=False)